# Model Tests

## General

### Imports

In [30]:
import pandas as pd
from sklearn import linear_model, model_selection, ensemble, tree
from pathlib import Path
import random

### Dataset

In [31]:
df = pd.read_csv(Path('./data/model/immoscout_robust.csv'))

df.head(10)

,ForestDensityM,Latitude,Longitude,NoisePollutionRailwayM,NoisePollutionRoadM,PopulationDensityM,RiversAndLakesM,RiversAndLakesS,distanceToTrainStation,gde_area_agriculture_percentage,...,type_penthouse,type_rustico,type_secondary-suite,type_semi-detached-house,type_single-room,type_stepped-apartment,type_stepped-house,type_studio,type_terrace-house,type_villa
0,1.067960,0.686833,0.130463,0.000000,-1.129839,0.010313,18.218171,0.011871,0.987703,-0.102607,...,1,0,0,0,0,0,0,0,0,0
1,1.067960,0.686833,0.130463,0.000000,-1.129839,0.010313,18.218171,0.011871,0.987703,-0.102607,...,0,0,0,0,0,0,0,0,1,0
2,0.215740,0.668920,0.106747,0.000000,0.715909,0.804150,1893.128856,0.000000,-0.071577,-0.709981,...,1,0,0,0,0,0,0,0,0,0
3,1.067960,0.686833,0.130463,0.000000,-1.129839,0.010313,18.218171,0.011871,0.987703,-0.102607,...,0,0,0,0,0,0,0,0,0,0
4,1.035876,0.676133,0.112097,0.000000,-0.742810,-0.302241,1422.882194,0.091805,0.202417,-0.025255,...,0,0,0,0,0,0,0,0,0,0
5,0.515181,0.672546,0.089462,0.000000,-0.682468,0.137930,0.000000,0.000000,1.042194,-0.029724,...,0,0,0,0,0,0,0,0,0,0
6,1.067960,0.686833,0.130463,0.000000,-1.129839,0.010313,18.218171,0.011871,0.987703,-0.102607,...,0,0,0,0,0,0,0,0,1,0
7,-0.213010,0.660603,0.106216,103.826366,0.677604,0.186520,0.000000,0.000000,-0.156544,-0.709981,...,0,0,0,0,0,0,0,0,0,0
8,1.747204,0.672320,0.122047,6.816164,0.882613,0.141751,391.719036,0.034402,-0.017196,-0.709981,...,0,0,0,0,0,0,0,0,0,0
9,0.413429,0.666867,0.089861,0.000000,-0.236134,0.007740,0.000000,0.000000,0.878246,-0.029724,...,0,0,0,0,0,0,1,0,0,0


### Helpers

In [32]:
def train_test_split(X, y):
    return model_selection.train_test_split(X, y, train_size=0.6, random_state=42)

In [33]:
def get_random_column_names(columns: list, percentage: float, add_type_columns: bool) -> list:
    type_columns = [c for c in columns if c.startswith("type_") and add_type_columns]
    regular_columns = [c for c in columns if (not c.startswith("type_")) and random.random() < percentage]
    return [*type_columns, *regular_columns]

## Model Functions

### Linear Models

In [34]:
# Linear Regression
def train_linear_regression(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, y_test: pd.Series) -> dict:
    model = linear_model.LinearRegression()
    model.fit(X_train, y_train)

    return {
        "type": "LinearRegression",
        "columns": list(X_train.columns),
        "num_columns": len(X_train.columns),
        "score": model.score(X_test, y_test),
        "model": model
    }

In [35]:
# Ridge
def train_ridge(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, y_test: pd.Series) -> dict:
    model = linear_model.Ridge()
    model.fit(X_train, y_train, 100)

    return {
        "type": "Ridge",
        "columns": list(X_train.columns),
        "num_columns": len(X_train.columns),
        "score": model.score(X_test, y_test),
        "model": model
    }

### Ensemble

In [36]:
# Random Forest
def train_random_forest(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, y_test: pd.Series) -> dict:
    model = ensemble.RandomForestRegressor()
    model.fit(X_train, y_train)

    return {
        "type": "RandomForest",
        "columns": list(X_train.columns),
        "num_columns": len(X_train.columns),
        "score": model.score(X_test, y_test),
        "model": model
    }

## Execution function

In [37]:
# Constants
NUM_ITERATIONS = 500
MIN_PERCENTAGE_COLUMNS = 0.2
MAX_PERCENTAGE_COLUMNS = 1
TRAINING_FUNCTIONS = [train_random_forest]

print("Total number of iterations:", NUM_ITERATIONS * len(TRAINING_FUNCTIONS) * 2)

Total number of iterations: 1000


In [38]:
# Data preparation
X = df.drop("price", axis=1)
y = df["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [39]:
results = []
for _ in range(NUM_ITERATIONS):
    for add_type_columns in [True, False]:
        column_percentage = random.random() * (MAX_PERCENTAGE_COLUMNS - MIN_PERCENTAGE_COLUMNS) + MIN_PERCENTAGE_COLUMNS
        column_names = get_random_column_names(X_train.columns, column_percentage, add_type_columns)
        temp_X_train, temp_X_test = X_train[column_names], X_test[column_names]

        for func in TRAINING_FUNCTIONS:
            results.append({**func(temp_X_train, temp_X_test, y_train, y_test), "add_type_columns": add_type_columns})

pd.DataFrame(results).sort_values("score", ascending=False)

ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values